<a href="https://colab.research.google.com/github/j-min/HiREST/blob/main/video_retrieval_inference_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference and Evaluation of video retrieval task on HiREST

* Project page: https://hirest-cvpr2023.github.io
* Code: https://github.com/j-min/HiREST

In [1]:
# Make sure GPU is used
!nvidia-smi

Thu Mar 30 02:17:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
git clone https://github.com/j-min/HiREST

In [ ]:
cd HiREST

# Install Packages

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git (to revision eff383b27b783e280c089475852ba83f20f64998) to /tmp/pip-req-build-yx0hmgd4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-yx0hmgd4
  Running command git rev-parse -q --verify 'sha^eff383b27b783e280c089475852ba83f20f64998'
  Running command git fetch -q https://github.com/openai/whisper.git eff383b27b783e280c089475852ba83f20f64998
  Running command git checkout -q eff383b27b783e280c089475852ba83f20f64998
  Resolved https://github.com/openai/whisper.git to commit eff383b27b783e280c089475852ba83f20f64998
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/bckim92/language-evaluation.git to /tmp/pip-req-build-u6wp_6t4
  Running command git clone --filter=blob:none --quiet https://github.com/bckim92/language-evaluation.git /tmp/pip-req-build-u6wp_6t4
  R

# Downloaded Pre-extracted features

In [5]:
# EVA-CLIP G features with 32 frames per video
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/eva_clip_features_32_frame.zip

--2023-03-30 02:28:01--  https://huggingface.co/j-min/HiREST-baseline/resolve/main/eva_clip_features_32_frame.zip
Resolving huggingface.co (huggingface.co)... 34.203.133.210, 52.2.178.255, 35.173.225.216, ...
Connecting to huggingface.co (huggingface.co)|34.203.133.210|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/e5/43e5477f8142d4d657272c2d83709064f52cea70061d7f463fe4e9973cc4707b/bdf94645389cad92d2659e3ff8378ff3389e4d2e56e6197ef988da47185cd9e4?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27eva_clip_features_32_frame.zip%3B+filename%3D%22eva_clip_features_32_frame.zip%22%3B&response-content-type=application%2Fzip&Expires=1680402483&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQzL2U1LzQzZTU0NzdmODE0MmQ0ZDY1NzI3MmMyZDgzNzA5MDY0ZjUyY2VhNzAwNjFkN2Y0NjNmZTRlOTk3M2NjNDcwN2IvYmRmOTQ2NDUzODljYWQ5MmQyNjU5ZTNmZjgzNzhmZjMzODllNGQyZTU2ZTYxOTdlZjk4OGRhNDcxODVjZ

In [6]:
!unzip -q eva_clip_features_32_frame.zip

In [7]:
!mv eva_clip_features_32_frame/ data/eva_clip_features/

# Download Pretrained Weights

In [8]:
!mkdir ./pretrained_weights/

In [9]:
# EVA-CLIP-G (frozen visual/textual encoder)
!wget https://huggingface.co/BAAI/EVA/resolve/main/eva_clip_psz14.pt
!mv eva_clip_psz14.pt ./pretrained_weights/eva_clip_psz14.pt

--2023-03-30 02:41:05--  https://huggingface.co/BAAI/EVA/resolve/main/eva_clip_psz14.pt
Resolving huggingface.co (huggingface.co)... 52.2.178.255, 54.82.45.103, 35.173.225.216, ...
Connecting to huggingface.co (huggingface.co)|52.2.178.255|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/f1/58f17426d51756a1aa3793fad0651594b6c14d7bf99c577181cb2e0614f61817/fe1d3a90bb88c3816a8c708b8a16c11c93cc323ec9e7bc7a0e8b979449fa9785?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27eva_clip_psz14.pt%3B+filename%3D%22eva_clip_psz14.pt%22%3B&Expires=1680403266&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU4L2YxLzU4ZjE3NDI2ZDUxNzU2YTFhYTM3OTNmYWQwNjUxNTk0YjZjMTRkN2JmOTljNTc3MTgxY2IyZTA2MTRmNjE4MTcvZmUxZDNhOTBiYjg4YzM4MTZhOGM3MDhiOGExNmMxMWM5M2NjMzIzZWM5ZTdiYzdhMGU4Yjk3OTQ0OWZhOTc4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFc

# Run video retrieval inference

You can change `n_model_frames` to other numbers <= 32.

In [13]:
!python inference_video_retrieval.py \
        --data_dir './data/splits' \
        --video_feature_dir './data/eva_clip_features' \
        --optim adamw \
        --n_model_frames 20 \
        --num_workers 4 \
        --eval_batch_size 10 \
        --device 'cuda' \
        --video_retrieval_model 'clip_g' \
        --run_name clip_g_VR_20frames_avgpool

<All keys matched successfully>
Loaded EVA CLIP G
self.videos: 1391
self.prompts: 546
split: test_negative_samples
self.videos: 2891
self.prompts: 2891
Number of prompts:  546
Number of videos:  4282
Computing text embeddings
Computing text embeddings: 100% 55/55 [00:03<00:00, 17.08it/s]
Text embeddings shape: torch.Size([546, 1024])
Computing video embeddings
Video feature dir: data/eva_clip_features
Video feature dir exists: True
N frames: 20
Computing video embeddings - N frames: 20: 100% 4282/4282 [01:04<00:00, 66.89it/s]
Video embeddings shape: torch.Size([4282, 1024])
Computing scores
Scores shape: torch.Size([546, 4282])
Preparing output json: 100% 546/546 [00:00<00:00, 6479.61it/s]
Saved results to VR_results/clip_g_VR_20frames_avgpool.json


# Calculate score

In [14]:
!python evaluate.py --task video_retrieval --pred_data VR_results/clip_g_VR_20frames_avgpool.json

Namespace(task='video_retrieval', gt_data=None, pred_data='VR_results/clip_g_VR_20frames_avgpool.json', device=-1, print_per_category=False, help=False, preprocess_moment_bounds=False, replace_pred_moment_bounds=False, frame_dir='None')
100% 546/546 [00:02<00:00, 203.67it/s]
{'total_prompt_count': 546, 'R@1': 26.373626373626376, 'R@5': 51.098901098901095, 'R@10': 61.53846153846154, 'R@50': 80.76923076923077}
